In [ ]:
import geopandas as gpd
import folium
import pandas as pd
from folium.plugins import FeatureGroupSubGroup
from branca.colormap import linear
from ipywidgets import interact
import numpy as np

shp_path = 'C:/Users/mathg/OneDrive/Documents/Diretório/Water/associação/MT/setores/MT_setores_CD2022.shp'
csv_folder = "C:/Users/mathg/OneDrive/Documents/Diretório/Water/associação/MT/Base informaçoes setores2010 universo MT/CSV"
col_csv = "Cod_setor"  # Chave primária

# Carregar o shapefile
gdf = gpd.read_file(shp_path)
gdf = gdf.rename(columns={"CD_SETOR": "Cod_setor"})  
gdf[col_csv] = gdf[col_csv].astype(str)
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Caminho da tabela do IBGE (CSV, Excel, etc.)
ibge_data_path = "C:/Users/mathg/OneDrive/Documents/Diretório/Water/associação/MT/Base informaçoes setores2010 universo MT/CSV/DomicilioRenda_MT.csv"

# Carregar a tabela com dados do IBGE
df_ibge = pd.read_csv(ibge_data_path, encoding='latin', sep=';')
df_ibge['Cod_setor'] = df_ibge['Cod_setor'].astype(str)

# Supondo que há uma coluna 'codigo_setor' em ambas as bases para fazer o merge
gdf = gdf.merge(df_ibge, on="Cod_setor", how="left")

colunas = df_ibge.columns

for coluna in colunas: 
    df_ibge[coluna] = df_ibge[coluna].replace('X', np.nan)
    df_ibge[coluna] = df_ibge[coluna].astype(float)

In [ ]:
import geopandas as gpd
import folium
import pandas as pd
from branca.colormap import linear
from ipywidgets import interact

# Unir os dados pelo código do setor
#gdf = gdf.merge(df_ibge, on="Cod_setor", how="left")

# Lista de variáveis disponíveis (somente numéricas)
variaveis = [col for col in df_ibge.columns if col != "codigo_setor"]

# Criar mapa base
def atualizar_mapa(variavel):
    m = folium.Map(
        location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()],
        zoom_start=12
    )

    # Garantir que a variável é numérica
    gdf[variavel] = pd.to_numeric(gdf[variavel], errors="coerce")

    # Criar colormap apenas se houver valores numéricos
    if gdf[variavel].dropna().shape[0] > 0:
        colormap = linear.YlGnBu_09.scale(
            gdf[variavel].min(skipna=True),
            gdf[variavel].max(skipna=True)
        )
    else:
        colormap = linear.YlGnBu_09.scale(0, 1)  # Evita erro caso todos os valores sejam NaN

    # Adicionar setores ao mapa
    for _, row in gdf.iterrows():
        valor = row[variavel]
        cor = colormap(valor) if pd.notna(valor) else "gray"

        folium.GeoJson(
            row["geometry"],
            style_function=lambda feature, cor=cor: {
                "fillColor": cor,
                "color": "black",
                "weight": 1,
                "fillOpacity": 0.7
            },
            tooltip=folium.Tooltip(f"{variavel}: {valor}")
        ).add_to(m)

    colormap.caption = variavel
    colormap.add_to(m)

    return m

interact(atualizar_mapa, variavel=variaveis)
